In [1]:
import pandas as pd
import numpy as np

Additional cleaning tasks:
* Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)
* Remove faults where the EquipmentID has more than 5 characters.

Upcoming full derate indicated by an SPN 5246 <-- requires a tow. 
SPN 1569, FMI = 31 75% derate

In [2]:
#Read in J1939Faults. low_memory = false because of warning about mixed type columns.
faults = pd.read_csv('../data/J1939Faults.csv',
                     index_col = 'RecordID',
                     parse_dates = ['EventTimeStamp', 'LocationTimeStamp'],
                     low_memory = False)
#remove faults that occur at service locations. Had to remove the last digit because the rows dont have it.
service_location1 = faults[(faults["Latitude"] == 36.066666) & (faults["Longitude"] == -86.434722)]

service_location2 = faults[(faults["Latitude"] == 35.588333) & (faults["Longitude"] == -86.443888)]

service_location3 = faults[(faults["Latitude"] == 36.1950) & (faults["Longitude"] ==  -83.174722)]

#concat service_locations for removal
service_locations = pd.concat([service_location1, service_location2, service_location3])
#drop service locations
faults_cleaned = faults.drop(service_locations.index)
#remove EquipmentIDs that are longer than 5 Characters per the README
faults_cleaned = faults_cleaned[faults_cleaned['EquipmentID'].map(len) <= 5]
#removed columns that only contained null values or would not be valuable because they're unique to the truck.
faults_cleaned = faults_cleaned.drop(['actionDescription', 'faultValue', 'ecuSerialNumber',
                                      'ecuSource', 'MCTNumber'], axis = 1).reset_index()
#consider keeping ecuSoftwareVersion, ecuModel, ecuMake. Maybe an updated software or particular make/model throws more codes

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#find the trucks that derate
derates = faults_cleaned[((faults_cleaned['spn'] == 1569) & (faults_cleaned['fmi'] == 31))| (faults_cleaned['spn'] == 5246)]

In [5]:
#get the EquipmentIDs of trucks that do eventually reach a derate.
derates['EquipmentID'].unique()

array(['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1547', '1444', '1480', '1466', '1457',
       '1794', '1696', '1703', '1626', '1715', '1437', '1453', '1584',
       '1597', '1610', '1377', '1652', '1517', '1784', '1751', '1786',
       '1590', '1341', '1382', '1714', '1711', '1369', '1398', '1791',
       '1772', '1472', '1418', '1645', '1475', '1815', '1806', '1607',
       

In [6]:
#create a list of trucks that reach a derate. this may make the EquipmentID <= 5 irrelevant since none of these
#are longer than 4 characters.
derates_list = ['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1547', '1444', '1480', '1466', '1457',
       '1794', '1696', '1703', '1626', '1715', '1437', '1453', '1584',
       '1597', '1610', '1377', '1652', '1517', '1784', '1751', '1786',
       '1590', '1341', '1382', '1714', '1711', '1369', '1398', '1791',
       '1772', '1472', '1418', '1645', '1475', '1815', '1806', '1607',
       '1560', '1450', '1401', '1572', '1375', '1368', '1742', '1650',
       '1443', '1495', '1670', '1587', '1778', '1566', '1617', '1691',
       '1573', '1477', '1476', '1524', '305', '1441', '1405', '1577',
       '1447', '1659', '1408', '1439', '1403', '1458', '1535', '1668',
       '1814', '1567', '1373', '1446', '1700', '1629', '1730', '1665',
       '1381', '1551', '1413', '1643', '1508', '1440', '1455', '1844',
       '1571', '1429', '1491', '1803', '1467', '1378', '1488', '1739',
       '1448', '1420', '1384', '1683', '304', '1372', '1502', '1582',
       '1464', '1599', '1712', '1407', '1603', '1644', '1757', '1618',
       '1604', '1519', '1831', '1631', '1811', '1628', '1818', '1423',
       '1636', '1555', '1719', '1638', '1710', '1823', '1435', '1808',
       '1392', '1669', '1649', '1579', '1416', '1534', '1575', '1397',
       '1539', '1510', '1813', '1601', '1657', '1621', '1651', '1620',
       '1708', '1592', '1613', '1619', '1925', '1887', '1662', '1640',
       '1881', '1563', '1713', '1557', '1839', '1833', '1605', '1513',
       '1639', '1866', '1485', '1581', '1549', '1776', '301', '308',
       '1586', '1661', '1635', '1907', '1853', '1785', '1594', '1859',
       '1600', '1963', '1848', '1918', '1561', '1796', '1624', '1598',
       '1972', '1706', '1641', '1874', '1940', '1609', '1642', '1506',
       '1552', '1685', '1883', '1946', '1913', '1920', '1634', '1995',
       '1512', '1878', '1564', '1686', '1554', '1505', '1802', '1788',
       '1868', '1807', '1688', '1922', '1903', '1681', '1529', '1904',
       '1615', '1828', '1694', '1559', '1827', '1745', '1842', '1720',
       '1525', '1970', '1877', '1888', '1989', '1537', '1932', '1521',
       '1804', '1854', '1851', '1565', '1981', '1528', '1741', '1980',
       '1737', '1580', '1924', '1822', '1766', '1812', '1917', '1933',
       '1817', '1654', '1731', '2011', '2017', '2008', '1726', '1717',
       '1656', '1543', '1835', '1664', '1880', '1916', '1996', '1912',
       '1947', '1997', '1736', '1820', '1729', '1864', '1556', '1898',
       '1884', '1832', '1705', '1585', '1908', '1752', '1623', '1800',
       '1732', '1725', '1845', '1876', '1576', '1897', '1647', '1931',
       '1971', '1852', '1697', '1959', '1953', '1787', '1962', '1893',
       '1746', '2015', '2019', '1957', '1593', '1965', '1873', '1911',
       '1934', '1942', '1982', '1900', '2009', '1948', '1723', '2006',
       '1956', '2012', '1836', '1935', '307', '1879', '1792', '1865',
       '1885', '1690', '1793', '1749', '1860', '1894', '1856', '1704',
       '1614', '1655', '1667', '1632', '1699', '1687', '1625', '1871',
       '1750', '1611', '2013', '1939', '1886', '1999', '1819', '1901',
       '1821', '1847', '1701', '1588', '1838', '1849', '1775', '1743',
       '1952', '1988', '1716', '1862', '1863', '1622', '2004', '1764',
       '1905', '1698', '1846', '309', '1798', '1780', '1762', '1979',
       '2007', '1919', '1966', '1843', '1938', '1744', '1992', '1976',
       '1926', '2143', '1718', '1899', '2111', '1978', '1763', '1870',
       '1889', '1994', '2027', '1967', '1928', '1921', '1891', '1734',
       '1748', '1945', '2010', '306', '1951', '1768', '1781', '2002',
       '1728', '1869', '1735', '1964', '1858', '1758', '1958', '2120',
       '1756', '2209', '1944', '1927', '2005', '1709', '2082', '1914',
       '2020', '1872', '1975', '1857', '1733', '1961', '1954', '1985',
       '1824', '1993', '1895', '1906', '2018', '1977', '1834', '1930',
       '1969', '1991', '1984', '1949', '1896', '2109', '2030', '1950',
       '1855', '2000', '1986', '1782', '1937', '2021', '1943', '2089',
       '1968', '1909', '2211', '1829', '302', '1861', '2175']
faults_cleaned = faults_cleaned[faults_cleaned['EquipmentID'].isin(derates_list)]

In [ ]:
#read in the Vehicle diagnostics file for informative engine characteristics and pivot to wide form.
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv') \
    .pivot(index = 'FaultId', columns = 'Name', values = 'Value')

#merge our cleaned faults_cleaned dataframe with diagnostics 
faults_diagnostics = faults_cleaned.merge(diagnostics, left_on = 'RecordID', right_on = 'FaultId') \
    .set_index('EventTimeStamp').sort_index()

#remove 'ServiceDistance' because they're all null. 
faults_diagnostics.drop(['ServiceDistance'], axis = 1).reset_index()

In [14]:
#create new csv of the cleaned and merged dataset.
faults_diagnostics.to_csv('../data/faults_diagnostics.csv')

In [8]:
faults_diagnostics.groupby('EquipmentID').head()

,RecordID,ESS_Id,eventDescription,ecuSoftwareVersion,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2000-03-18 19:14:10,1211418,108604426,High Voltage (Fuel Level),NaN,CECU3B-NAMUX4,PACCR,96,3,True,126,...,NaN,True,127.4,1279,False,NaN,0,NaN,100,0.58
2000-03-18 19:14:10,1211417,108604425,High Voltage (Left Fuel Level Sensor),NaN,CECU3B-NAMUX4,PACCR,829,3,True,126,...,NaN,True,127.4,1279,False,NaN,0,NaN,100,0.58
2000-03-18 19:20:47,1211420,108604488,High Voltage (Left Fuel Level Sensor),NaN,CECU3B-NAMUX4,PACCR,829,3,False,126,...,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-18 19:20:47,1211419,108604487,High Voltage (Fuel Level),NaN,CECU3B-NAMUX4,PACCR,96,3,False,126,...,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-19 02:59:58,1211422,108608408,Not Reporting Data Wheel Sensor ABS Axle 2 Right,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,EC80ESP,BNDWS,792,7,False,13,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-05 10:39:48,1075527,67316888,Low (Severity Medium) Engine Coolant Level,04384413*22246857*121817205924*60701721*G1*BGT*,6X1u17D1500000000,CMMNS,111,18,True,1,...,NaN,True,143.6,2047,False,NaN,1.203907,NaN,100,0.29
2018-11-05 10:41:27,1075530,67316946,Low (Severity Medium) Engine Coolant Level,04384413*22246857*121817205924*60701721*G1*BGT*,6X1u17D1500000000,CMMNS,111,18,False,1,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-18 16:42:31,1108957,74673517,Low (Severity Medium) Transmission Air Tank Pr...,5516010*202.21.0*5516502*E003.e003*5539401*25....,EEO-xxF112C,EATON,37,18,True,3,...,NaN,True,129.2,50175,False,NaN,0,NaN,100,0.29


In [9]:
faults_diagnostics[faults_diagnostics['spn']== 5246]

,RecordID,ESS_Id,eventDescription,ecuSoftwareVersion,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:03:22,1237840,119019218,NaN,04384413*22115617*090617144354*60701715*G1*BGT*,6X1u17D1500000000,CMMNS,5246,16,True,1,...,NaN,True,93.2,6143,True,NaN,0,NaN,100,0
2011-01-01 00:03:23,1237841,119019219,NaN,04384413*22115617*090617144354*60701715*G1*BGT*,6X1u17D1500000000,CMMNS,5246,0,True,1,...,NaN,True,93.2,6143,True,NaN,0,NaN,100,0
2011-01-01 00:03:34,1120933,77308358,NaN,05317106*04101156*020516150530*09400053*G1*BDR*,6X1u13D1500000000,CMMNS,5246,0,True,1,...,32,True,96.8,22527,False,NaN,9.689507,NaN,100,1.45
2011-01-01 00:04:57,378085,7367914,NaN,04993120*00041643*042114185815*07700062*I0*BBZ*,6X1u10D1500000000,CMMNS,5246,0,True,1,...,NaN,True,77,22527,True,NaN,0,NaN,NaN,0
2011-01-01 00:16:06,378082,7367911,NaN,04993120*00041643*042114185815*07700062*I0*BBZ*,6X1u10D1500000000,CMMNS,5246,0,True,1,...,NaN,True,102.2,22527,NaN,NaN,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-19 07:02:33,1244156,121610128,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,6X1u13D1500000000,CMMNS,5246,0,True,1,...,NaN,True,41,22527,NaN,NaN,0,NaN,100,0.58
2020-02-21 07:23:44,1244808,121909497,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,6X1u17D1500000000,CMMNS,5246,16,True,1,...,NaN,True,50,18431,True,NaN,0,NaN,100,0
2020-02-24 15:27:26,1245454,122305096,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,6X1u17D1500000000,CMMNS,5246,0,True,1,...,NaN,True,55.4,5119,True,NaN,0,NaN,100,0


In [10]:
service_code = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
service_code[service_code['SPN']== 5246]

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
2518,Y,3712,167,Not Mapped,Not Mapped,Not Mapped,0,5246,0,Not Mapped,Red,Stop / Shutdown,Aftertreatment SCR Operator Inducement - Data ...,SCR inducement of 5 mph derate - Fault Code 41...
2781,Y,4134,167,Not Mapped,Not Mapped,Not Mapped,0,5246,15,Not Mapped,Amber,Warning,Aftertreatment SCR Operator Inducement - Data ...,SCR inducement - Least Severe - Fault Code 371...
4338,Y,6254,167,Not Mapped,Not Mapped,Not Mapped,0,5246,16,Not Mapped,Amber,Warning,Aftertreatment SCR Operator Inducement Severit...,NaN


In [12]:
service_code[service_code['SPN']== 1569]

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
2520,Y,3714,167,Not Mapped,Not Mapped,Not Mapped,11,1569,31,Not Mapped,Amber,Warning,Engine Protection Torque Derate - Condition Ex...,NaN
4339,Y,6255,167,Not Mapped,Not Mapped,Not Mapped,0,1569,15,Not Mapped,None,None,Engine Protection Torque Derate - Data Valid B...,NaN
5095,Y,7285,167,Not Mapped,Not Mapped,Not Mapped,14,1569,14,Not Mapped,Amber,Warning,Engine Protection Torque Derate - Special Inst...,NaN


In [13]:
vehicle_make = pd.read_excel('../data/Vehicle_Make.xlsx')